<a href="https://colab.research.google.com/github/hyomee2/scooter-parking-detector/blob/main/models/yolov8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1. 구글 드라이브 마운트

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## 2. YOLOv8을 위한 ultralytics 설치

In [ ]:
!pip install ultralytics

## 3. YOLOv8을 위한 데이터 디텍토리 구조 생성

In [ ]:
import os
import shutil
import random
from pathlib import Path

# 경로 설정
RAW_DIR = Path('/content/drive/MyDrive/데기인 팀플 7조/final_output/dataset/raw')
LABEL_DIR = Path('/content/drive/MyDrive/데기인 팀플 7조/final_output/dataset/labelled_data')
YOLO_DIR = Path('/content/drive/MyDrive/데기인 팀플 7조/final_output/dataset/yolo')
CLASSES = ['improper', 'noise', 'proper']
SPLITS = ['train', 'val', 'test']
SPLIT_RATIO = [0.8, 0.1, 0.1]

# 기존 YOLO 하위 디렉토리 삭제 후 생성
for split in SPLITS:
    split_dir = YOLO_DIR / split
    if split_dir.exists():
        shutil.rmtree(split_dir)  # 기존 폴더 통째로 삭제
    (split_dir / 'images').mkdir(parents=True, exist_ok=True)
    (split_dir / 'labels').mkdir(parents=True, exist_ok=True)

def split_and_copy_images():
    for cls in CLASSES:
        # 디렉터리 무시하고 파일만 필터링
        image_paths = [p for p in (RAW_DIR / cls).glob('*.*') if p.is_file()]
        random.shuffle(image_paths)

        total = len(image_paths)
        train_end = int(total * SPLIT_RATIO[0])
        val_end = train_end + int(total * SPLIT_RATIO[1])

        split_paths = {
            'train': image_paths[:train_end],
            'val': image_paths[train_end:val_end],
            'test': image_paths[val_end:]
        }

        for split, paths in split_paths.items():
            for img_path in paths:
                # 이미지 복사
                dst_img_path = YOLO_DIR / split / 'images' / img_path.name
                shutil.copy(img_path, dst_img_path)

                # 레이블 복사
                label_name = img_path.with_suffix('.txt').name
                label_src = LABEL_DIR / label_name
                label_dst = YOLO_DIR / split / 'labels' / label_name
                if label_src.exists():
                    shutil.copy(label_src, label_dst)
                else:
                    print(f'[경고] 레이블 파일 없음: {label_src}')

if __name__ == "__main__":
    split_and_copy_images()
    print("✅ 데이터 분할 및 복사 완료.")


In [ ]:
import yaml

def create_data_yaml():
    yaml_dict = {
        'train': str(YOLO_DIR / 'train' / 'images'),
        'val': str(YOLO_DIR / 'val' / 'images'),
        'nc': len(CLASSES),
        'names': CLASSES
    }
    yaml_path = YOLO_DIR / 'data.yaml'
    with open(yaml_path, 'w') as f:
        yaml.dump(yaml_dict, f)
    print(f"✅ data.yaml 생성 완료 → {yaml_path}")

if __name__ == "__main__":
    split_and_copy_images()
    create_data_yaml()  # yaml 파일 생성까지 추가
    print("✅ 데이터 분할 + yaml 파일까지 생성 완료.")

## 4. YOLOv8을 이용한 훈련 실행

In [ ]:
from ultralytics import YOLO
# 모델 로드 (사전 학습된 모델 사용)
model = YOLO('yolov8n.pt')

In [ ]:
import torch
yaml_path = YOLO_DIR / 'data.yaml'
# 훈련 실행
model.train(
    data=yaml_path,  # data.yaml 파일 경로
    epochs=300,  # 훈련 epoch 수
    patience = 30,
    imgsz=640,  # 이미지 크기 (가장 일반적인 크기가 640)
    batch=32,  # 배치 크기

    # Learning rate schedule
    lr0=0.02,      # 초기 learning rate → 약간 높게 시작해서 빠른 탐색 유도
    lrf=0.01,      # 최종 learning rate multiplier (0.02 * 0.01 = 0.0002 수준까지 감소)
    warmup_epochs=5,


    # 증강
    # degrees=15.0, # ±15도 내외 회전 (= Rotation_range=15)
    translate=0.1, # 수평/수직 이동 (각각 이미지 크기의 10%) (= width_shift_range=0.1 & height_shift_range=0.1)
    fliplr=1.0, # 좌우 반전 (horizontal_flip=True)
    scale=0.2, # 확대/축소 (20% 내외로 확대/축소) (= zoom_range=0.2 )
    hsv_v=0.2, # 밝기 변화 조정 (brightness_range=[0.8, 1.2]과 비슷하게 맞춤)
    augment=True,        # 기본 augment 활성화
    mosaic=0.3,
    mixup=0.2,

    project='/content/drive/MyDrive/result',  # 결과 저장 폴더
    name='yolov8n_finetune',    # 훈련 이름
    pretrained=True,            # 사전 학습된 모델 사용
    val=True                    # 검증 수행
)

## 5. YOLO 모델 평가

In [ ]:
import pandas as pd
result = pd.read_csv('/content/drive/MyDrive/result/yolov8n_finetune5/results.csv')

In [ ]:
import matplotlib.pyplot as plt
# 시각화할 주요 지표
metrics_to_plot = [
    ('train/box_loss', 'val/box_loss'),
    ('train/cls_loss', 'val/cls_loss'),
    ('train/dfl_loss', 'val/dfl_loss'),
    ('metrics/mAP50(B)', 'metrics/mAP50-95(B)')
]

# 그래프 그리기
for train_metric, val_metric in metrics_to_plot:
    if train_metric in result.columns and val_metric in result.columns:
        plt.figure()
        plt.plot(result[train_metric], label=train_metric)
        plt.plot(result[val_metric], label=val_metric)
        plt.xlabel('Epoch')
        plt.ylabel('Loss/Metric')
        plt.title(f'{train_metric} vs {val_metric}')
        plt.legend()
        plt.grid(True)
        plt.tight_layout()
        plt.show()
    else:
        print(f"❌ 열이 존재하지 않습니다: {train_metric} 또는 {val_metric}")